In [4]:
!PIP install drive

     -------------------------------------- 98.2/98.2 kB 935.8 kB/s eta 0:00:00
     ---------------------------------------- 10.7/10.7 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB 5.8 MB/s eta 0:00:00
     -------------------------------------- 120.3/120.3 kB 7.3 MB/s eta 0:00:00
     ------------------------------------- 218.0/218.0 kB 13.9 MB/s eta 0:00:00


In [8]:
!pip install opencv-python

  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)


In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical

import cv2
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
#from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

In [2]:
df = pd.read_csv('transformation.csv')

In [4]:
#簡單的
x, y = df[df.columns[1:]], df[df.columns[0]]
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

x_traval, x_test, y_traval, y_test = train_test_split(x, y, test_size=0.2, random_state = 1, stratify= y)
x_train, x_val, y_train, y_val = train_test_split(x_traval, y_traval, test_size=0.2, random_state = 1, stratify= y_traval)



In [5]:
#reshape
x_train_r = np.array(x_train).reshape(-1,100,100)
x_test_r = np.array(x_test).reshape(-1,100,100)
x_val_r = np.array(x_val).reshape(-1,100,100)

# y_train = np.array(y_train)
# y_test = np.array(y_test)
# y_val = np.array(y_val)

# 轉換色彩 0~255 資料為 0~1
x_train_r = x_train_r.astype('float32')
x_val_r = x_val_r.astype('float32')
x_test_r = x_test_r.astype('float32')

x_train_r /= 255
x_val_r /= 255
x_test_r /= 255

# y 值轉成 one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes=50, dtype='float32') 
y_test = keras.utils.to_categorical(y_test, num_classes=50, dtype='float32')
y_val = keras.utils.to_categorical(y_val, num_classes=50, dtype='float32')

# mnist 的圖集黑色是255，白色是 0，而圖片讀進來是 黑色是0，白色是 255
# 所以在這裡除255後，也順便把黑白翻轉過來。
#data = 1 - data / 255.0 已經做了uint8

In [ ]:
#標準化 min-max https://stackoverflow.com/questions/72517783/scaling-row-wise-with-minmaxscaler-from-sklearn

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator #利用現有的資料經過旋轉、翻轉、縮放…等方式增加更多的訓練資料
# train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
# test_datagen = ImageDataGenerator(rescale = 1./255)


# #要把資料做轉換的步驟則是用 .flow(X,y) 或是 .flow_from_directory(directory) 讀取資料集+批量生成器，產生每epoch訓練樣本
# training_set = train_datagen.flow_from_directory("./classify/dataset/training_set", target_size = (64, 64), batch_size = 5)
# test_set = test_datagen.flow_from_directory("./classify/dataset/test_set", target_size = (64, 64), batch_size = 5)


In [8]:
# 定義分類數量
num_classes = 50

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 1), activation='relu')) #32 is no. of filters and kernel size is 5*5. ReLU is activation layer
model.add(MaxPooling2D(pool_size=(2,2))) #add Max pooling layer with kernel size 2*2 
# model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), input_shape=(100, 100, 1), activation='relu')) #32 is no. of filters and kernel size is 5*5. ReLU is activation layer
model.add(MaxPooling2D(pool_size=(2,2))) #add Max pooling layer with kernel size 2*2 
model.add(Conv2D(128, (3, 3), input_shape=(100, 100, 1), activation='relu')) #32 is no. of filters and kernel size is 5*5. ReLU is activation layer
model.add(MaxPooling2D(pool_size=(2,2))) #add Max pooling layer with kernel size 2*2 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax')) 

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x_train_r, y_train, validation_data=(x_val_r, y_val), epochs=20, batch_size=200)

#Evaluating model in keras
score = model.evaluate(x_test_r, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# model.fit_generator(training_set, epochs = 2, validation_data = test_set, verbose = 1)
# score = model.evaluate_generator(test_set)


#model.save('EfficientNetB0-40-Stripped.h5', include_optimizer=False)如果你對這個模型滿意，想要保留之後使用的話，可以這樣設定儲存參數，那麼優化器的狀態不會被保存下來，可以節省不少體積，減少的體積量依使用優化器的不同而定，使用adam的話，這麼做是很有感的，如果你是之前中斷訓練，且有意後續載入繼續進行訓練的話，建議可以先不做。

Epoch 1/20
3/3 [==============================] - 7s 2s/step - loss: 3.9835 - acc: 0.0098 - val_loss: 3.9140 - val_acc: 0.0156
Epoch 2/20
3/3 [==============================] - 4s 1s/step - loss: 3.9121 - acc: 0.0156 - val_loss: 3.9120 - val_acc: 0.0312
Epoch 3/20
3/3 [==============================] - 4s 1s/step - loss: 3.9116 - acc: 0.0254 - val_loss: 3.9130 - val_acc: 0.0312
Epoch 4/20
3/3 [==============================] - 5s 2s/step - loss: 3.9087 - acc: 0.0215 - val_loss: 3.9157 - val_acc: 0.0000e+00
Epoch 5/20
3/3 [==============================] - 4s 1s/step - loss: 3.9052 - acc: 0.0254 - val_loss: 3.9181 - val_acc: 0.0156
Epoch 6/20
3/3 [==============================] - 4s 1s/step - loss: 3.8987 - acc: 0.0293 - val_loss: 3.9219 - val_acc: 0.0234
Epoch 7/20
3/3 [==============================] - 4s 1s/step - loss: 3.8859 - acc: 0.0410 - val_loss: 3.9288 - val_acc: 0.0156
Epoch 8/20
3/3 [==============================] - 5s 1s/step - loss: 3.8675 - acc: 0.0273 - val_loss: 3.937